# Selecting imaging and pathology reports for labelling


Andres Tamm

2022-08-31


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-reports-for-TNM-and-recurrence" data-toc-modified-id="Prepare-reports-for-TNM-and-recurrence-1">Prepare reports for TNM and recurrence</a></span></li><li><span><a href="#Select-reports-for-TNM-and-recurrence" data-toc-modified-id="Select-reports-for-TNM-and-recurrence-2">Select reports for TNM and recurrence</a></span><ul class="toc-item"><li><span><a href="#Select-reports-for-TNM" data-toc-modified-id="Select-reports-for-TNM-2.1">Select reports for TNM</a></span></li><li><span><a href="#Recurrence" data-toc-modified-id="Recurrence-2.2">Recurrence</a></span></li></ul></li><li><span><a href="#Additional.-How-many-reports-to-select?" data-toc-modified-id="Additional.-How-many-reports-to-select?-3">Additional. How many reports to select?</a>

In [ ]:
import os
import pandas as pd
import numpy as np
import importlib as imp
import regex as re
import matplotlib.pyplot as plt
from scipy.stats import norm, truncnorm
from itertools import product
from time import sleep
from IPython.display import clear_output
from datetime import datetime
import textmining.utils as ut
from textmining.recurrence import get_recurrence
from textmining.reports import get_crc_reports
from textmining.tnm.clean import add_tumour_tnm
from textmining.tnm.tnm import get_tnm_phrase, get_tnm_values
from textmining.crm_emvi import get_crm, get_emvi
from pathlib import Path

In [ ]:
# Check current directory
os.getcwd()

In [ ]:
# Paths
root = Path("z:\\Andres\\project_textmining\\textmining\\")
code_path = root
out_path  = root / 'labelled_data'
data_path = root / 'data'

print(out_path.exists())
print(data_path.exists())

In [ ]:
os.listdir(data_path)

## 1. Prepare reports for TNM and recurrence

### 1.1. Gather reports

In [ ]:
# Get imaging types of interest
img_path = code_path / "textmining" / "vocab" / "NIHR-HIC_Colorectal-Cancer_imaging-types.xlsx"
img = pd.read_excel(img_path)
img = img.loc[~img['needed for NIHR HIC CRC'].isin(['no', 'maybe'])]

pd.set_option('display.max_colwidth', 500, 'display.min_rows', 10000, 'display.max_rows', 10000)
display(img)

codes = img.imaging_code.to_list()
# Not including 'ZRMDT','ZRXTC','ZRXTFL','ZRXTI','ZRXTM','ZRXTN','ZRXTP','ZRXTUS','ZSECOP' - tend to be empty
print(codes)

In [ ]:
## Get older OUH and RMH reports

# Files
files_img  = ['imaging_OUH_2022-08-17_144307.csv', 'imaging_RMH_2022-04-25_111348.csv']
files_path = ['histopathology_OUH_2022-04-24_203630.csv', 'histopathology_RMH_2022-04-25_111342.csv']
files_end = ['endoscopy_reports_OUH_2022-04-24_202454.csv']

df = pd.DataFrame()

# Imaging reports
for f in files_img:
    print('\n----Reading data from: {}'.format(f))
    t = pd.read_csv(data_path / f)    
    print('\nColumns: {}'.format(t.columns.to_list()))
    print('\nShape of data: {}'.format(t.shape))
    print('\nImaging code is available for {}% of reports'.format((~t.imaging_code.isna()).mean()*100))
    
    # Imaging codes
    c = t.imaging_code.value_counts()
    print('\nTop value counts of imaging codes: \n{}'.format(c[0:10]))
    mask = t.imaging_code.fillna('').str.lower().str.contains('|'.join(codes).lower(), regex=True)
    t = t.loc[mask]
    print('\nShape of data after including relevant img reports: {}'.format(t.shape))  
    
    t = t[['brc', 'subject_id', 'imaging_date', 'imaging_report_date', 'imaging_code', 'report_text_anon']]
    t = t.rename(columns={'imaging_report_date':'report_date'})
    t['report_type'] = 'imaging'

    print(t.report_date.iloc[0:5])
    t.report_date = pd.to_datetime(t.report_date)

    df = pd.concat(objs=[df,t], axis=0)

# Pathology reports
for f in files_path:
    print('\n----Reading data from: {}'.format(f))
    t = pd.read_csv(data_path / f)    
    print('\nColumns: {}'.format(t.columns.to_list()))
    print('\nShape of data: {}'.format(t.shape))

    t = t[['brc', 'subject_id', 'authorised_date', 'report_text_anon']]
    t = t.rename(columns={'authorised_date':'report_date'})
    t['report_type'] = 'pathology'

    print(t.report_date.iloc[0:5])
    t.report_date = pd.to_datetime(t.report_date)

    df = pd.concat(objs=[df,t], axis=0)
    
# Endoscopy reports
for f in files_end:
    print('\n----Reading data from: {}'.format(f))
    t = pd.read_csv(data_path / f)    
    print('\nColumns: {}'.format(t.columns.to_list()))
    print('\nShape of data: {}'.format(t.shape))
    
    t = t[['brc', 'subject_id', 'exam_type', 'exam_date', 'report_text_anon']]
    t = t.rename(columns={'exam_date':'report_date'})
    t = t.loc[t.exam_type.str.lower().str.contains('colonos|sigmoidos')]
    t['report_type'] = 'endoscopy'

    print(t.report_date.iloc[0:5])
    t.report_date = pd.to_datetime(t.report_date)

    df = pd.concat(objs=[df,t], axis=0)
    
# Some reports have duplicate texts - drop
print('Shape before dropping duplicates: {}'.format(df.shape[0]))
df = df.drop_duplicates(subset=['report_text_anon'])
print('Shape after dropping duplicates: {}'.format(df.shape[0]))

# Replace \r and ” with \n\n, and 
#  Not having \r simplifies reading csv from file (Pandas has issues otherwise)
#  ” seems to mark sections in RMH reports and having \n\n simplifies viewing
df.report_text_anon = df.report_text_anon.str.replace('\r', '\n\n')
df.report_text_anon = df.report_text_anon.str.replace('”', '\n\n')

# Reset index
df = df.reset_index(drop=True)

# Summary
print('\n====SUMMARY====\n\n{}'.format(df.groupby(['brc', 'report_type']).size()))
print('\nTotal number of reports: {}'.format(df.shape[0]))
print('\nColumns: {}'.format(df.columns.to_list()))

In [ ]:
# Dbl check report date range --> potential to include future reports as test set
df['report_date'] = pd.to_datetime(df['report_date'])
s = df.groupby(['brc', 'report_type'])['report_date'].agg([np.min, np.max])
print(s)

#t = df.loc[df.brc=='OXFORD']
#print(t.report_date.dt.year.value_counts().reset_index().sort_values(by='index'))

In [ ]:
# ---- Identify individuals with CRC in newer OUH data ----
f0 = data_path / '20220526.vw_outpatient_attendances_diagnoses.csv'
d0 = pd.read_csv(f0, usecols=['brc', 'subject', 'diagnosis_code_icd10', 'diagnosis_date'])
d0['src'] = 'outpat'

f1 = data_path / '20220526.vw_inpat_episodes_diagnoses.csv'
d1 = pd.read_csv(f1, usecols=['brc', 'subject', 'diagnosis_code_icd10', 'diagnosis_date'])
d1['src'] = 'inpat'

d = pd.concat(objs=[d0, d1], axis=0)
print(d.shape)

print(d.diagnosis_date.iloc[0:5])
d.diagnosis_date = pd.to_datetime(d.diagnosis_date)
print(d.diagnosis_date.iloc[0:5])
print(d.diagnosis_date.min(), d.diagnosis_date.max())
print(d.diagnosis_date.sort_values().drop_duplicates())

d = d.loc[d.diagnosis_code_icd10.fillna('').str.lower().str.contains('^c(?:18|19|20)', regex=True), :]
print(d.diagnosis_code_icd10.unique())
print(d.groupby('src')['subject'].nunique())

crc = d.subject.unique()
print(len(crc))


In [ ]:
# ---- Add newer OUH pathology reports for individuals with CRC ----
f = data_path / '20220526.vw_pathology_reports.csv'
#f = data_path / '20230329.vw_pathology_reports.csv'

t = pd.read_csv(f)
print(t.shape, t.columns)
t = t.rename(columns={'patient_id': 'subject_id', 'date_received': 'report_date', 'safe_report': 'report_text_anon'})
t = t.drop(labels=['calc_lab_no', 'date_authorised', 'snomed_t', 'snomed_m'], axis=1)
print(t.shape, t.columns)

print(t.report_date.iloc[0:5])
t.report_date = pd.to_datetime(t.report_date, format='%d/%m/%Y %H:%M:%S')
print(t.report_date.iloc[0:5])
print(t.report_date.min(), t.report_date.max())

t = t.loc[t.report_date >= '2021-01-01']
#t = t.loc[t.report_date >= '2022-03-31']
print(t.shape)

t['brc'] = 'OXFORD'
t['report_type'] = 'pathology_future'

t = t.loc[t.subject_id.isin(crc)]
print(t.shape)

df = pd.concat(objs=[df, t], axis=0)


In [ ]:
df.shape

In [ ]:
# Tmp - compare old vs new path report data
dfs = []

for f in [data_path / '20220526.vw_pathology_reports.csv',
          data_path / '20230329.vw_pathology_reports.csv']:

    t = pd.read_csv(f)
    print(t.shape, t.columns)
    t = t.rename(columns={'patient_id': 'subject_id', 'date_received': 'report_date', 'safe_report': 'report_text_anon'})
    t = t.drop(labels=['calc_lab_no', 'date_authorised', 'snomed_t', 'snomed_m'], axis=1)
    print(t.shape, t.columns)

    print(t.report_date.iloc[0:5])
    t.report_date = pd.to_datetime(t.report_date, format='%d/%m/%Y %H:%M:%S')
    print(t.report_date.iloc[0:5])
    print(t.report_date.min(), t.report_date.max())

    #t = t.loc[t.report_date >= '2021-01-01']
    #t = t.loc[t.report_date >= '2022-03-31']
    print(t.shape)

    t['brc'] = 'OXFORD'
    t['report_type'] = 'pathology_future'

    dfs.append(t)


In [ ]:
# Tmp - compare old vs new path report data
# Seems there are very few new path reports ... maybe because most patients already had major operation?
for t in dfs:
    print('--')
    print(t.shape)
    print(t.report_date.min(), t.report_date.max())

In [ ]:
# ---- Add newer OUH imaging reports for individuals with CRC ----
f = '20220526.vw_imaging.csv'
#f = '20230329.vw_imaging.csv'
print('\n----Reading data from: {}'.format(f))
t = pd.read_csv(data_path / f)    
print('\nColumns: {}'.format(t.columns.to_list()))
print('\nShape of data: {}'.format(t.shape))
print('\nImaging code is available for {}% of reports'.format((~t.imaging_code.isna()).mean()*100))

# Imaging codes
c = t.imaging_code.value_counts()
print('\nTop value counts of imaging codes: \n{}'.format(c[0:10]))
mask = t.imaging_code.fillna('').str.lower().str.contains('|'.join(codes).lower(), regex=True)
t = t.loc[mask]
print('\nShape of data after including relevant img reports: {}'.format(t.shape))  

# Reformat
t = t[['brc', 'subject', 'imaging_date', 'imaging_report_date', 'imaging_code', 'anonymised_report']]
t = t.rename(columns={'imaging_report_date':'report_date', 'subject': 'subject_id', 
                      'anonymised_report': 'report_text_anon'})
t['report_type'] = 'imaging_future'

# Date range
print(t.report_date.iloc[0:5])
t.report_date = pd.to_datetime(t.report_date, format='%d/%m/%Y %H:%M:%S')
print(t.report_date.iloc[0:5])
print(t.report_date.min(), t.report_date.max())
t = t.loc[t.report_date >= '2021-01-01']
#t = t.loc[t.report_date >= '2022-03-01']
print(t.shape)

# Retain CRC
t = t.loc[t.subject_id.isin(crc)]
print(t.shape)


df = pd.concat(objs=[df, t], axis=0)

In [ ]:
# tmp - compare old vs new imaging reports
dfs = []
for f in ['20220526.vw_imaging.csv', '20230329.vw_imaging.csv']:
    print('\n----Reading data from: {}'.format(f))
    t = pd.read_csv(data_path / f)    
    print('\nColumns: {}'.format(t.columns.to_list()))
    print('\nShape of data: {}'.format(t.shape))
    print('\nImaging code is available for {}% of reports'.format((~t.imaging_code.isna()).mean()*100))

    # Imaging codes
    c = t.imaging_code.value_counts()
    print('\nTop value counts of imaging codes: \n{}'.format(c[0:10]))
    mask = t.imaging_code.fillna('').str.lower().str.contains('|'.join(codes).lower(), regex=True)
    t = t.loc[mask]
    print('\nShape of data after including relevant img reports: {}'.format(t.shape))  

    # Reformat
    t = t[['brc', 'subject', 'imaging_date', 'imaging_report_date', 'imaging_code', 'anonymised_report']]
    t = t.rename(columns={'imaging_report_date':'report_date', 'subject': 'subject_id', 
                        'anonymised_report': 'report_text_anon'})
    t['report_type'] = 'imaging_future'

    # Date range
    print(t.report_date.iloc[0:5])
    t.report_date = pd.to_datetime(t.report_date, format='%d/%m/%Y %H:%M:%S')
    print(t.report_date.iloc[0:5])
    print(t.report_date.min(), t.report_date.max())
    #t = t.loc[t.report_date >= '2021-01-01']
    #t = t.loc[t.report_date >= '2022-03-01']
    print(t.shape)

    dfs.append(t)

In [ ]:
# Tmp - compare old vs new img report data
# New imaging table has more reports but same max date 
# -- perhaps due to similar issue as with HIC data, where some img 
for t in dfs:
    print('--')
    print(t.shape)
    print(t.report_date.min(), t.report_date.max())

In [ ]:
# Check count
df.groupby(['brc', 'report_type']).size()

In [ ]:
# Drop reports with duplicate text
print(df.shape[0], df.report_text_anon.nunique(), df.drop_duplicates().shape[0])

df = df.drop_duplicates(subset=['report_text_anon'])
print(df.shape[0])

In [ ]:
# Check count again
df.groupby(['brc', 'report_type']).size()

In [ ]:
# Check date range again
df['report_date'] = pd.to_datetime(df['report_date'])
s = df.groupby(['brc', 'report_type'])['report_date'].agg([np.min, np.max])
print(s)

In [ ]:
# Check reports with imaging codes starting with Z
#tmp = df.loc[df.imaging_code.fillna('').str.contains('^Z')].report_text_anon.drop_duplicates()
#tmp

In [ ]:
# For quick testing
#df = df.sample(10).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
# Save 
df.to_csv(out_path / 'all_reports.csv', index=False)

### 1.2. Run NLP

In [ ]:
# Read reports 
df = pd.read_csv(out_path / 'all_reports.csv')
print(df.shape[0], df.columns)

# Date to datetime
print(df.report_date.iloc[0:5])
df.report_date = pd.to_datetime(df.report_date)

# Use only small number of reports? For testing
testmode = False
if testmode:
    df = df.sample(100, random_state=42)

# Check count
df.groupby(['brc', 'report_type']).size()

In [ ]:
# Find reports that describe current colorectal cancer, but do not remove non-crc reports (ran about 43 minutes for 74k reports)
__, matches_crc = get_crc_reports(df, 'report_text_anon', add_subj_to_matches=True, subjcol='subject_id')

df['row'] = np.arange(df.shape[0])
df['crc_nlp'] = 0
matches_incl = matches_crc.loc[matches_crc.exclusion_indicator==0]
df.loc[df.row.isin(matches_incl.row), 'crc_nlp'] = 1
print(df.groupby(['brc', 'report_type'])['crc_nlp'].sum())

# Identify reports where all matches for CRC were marked as false 
# This helps check whether some cases of CRC may be completely missed when using the code
# As otherwise, a report could be marked as describing CRC if it has at least one valid match
df['row'] = np.arange(df.shape[0])
df['false_crc_nlp'] = 0
matches_excl = matches_crc.loc[matches_crc.exclusion_indicator==1]
row_false = np.setdiff1d(matches_excl.row, matches_incl.row)
df.loc[df.row.isin(row_false), 'false_crc_nlp'] = 1
print(df.groupby(['brc', 'report_type'])['false_crc_nlp'].sum())

# Save to disk
os.chdir(out_path)
#tstamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')

fname = 'matches-crc.csv'
print('\nSaving matches to file {}...'.format(fname))
matches_crc.to_csv(fname, index=False)

fname = 'reports-all_crc-true_tnm-false_recur-false.csv'
print('\nSaving reports to file {}...'.format(fname))
df.to_csv(fname, index=False)

In [ ]:
# Read reports (with CRC status)
read_from_disk=False
if read_from_disk:
    os.chdir(out_path)
    files = os.listdir()
    fname = [f for f in files if f.startswith('reports-all_crc-true_tnm-false_recur-false')][0]
    print(fname)
    df = pd.read_csv(fname)
    print(df.crc_nlp.mean())
    display(df.head())

In [ ]:
# Extract TNM phrases (ran about 163 minutes for 74k reports)
# TNM phrases marked as historical are not removed - could be removed later, and value extraction rerun
matches_tnm, check_phrases_tnm, check_cleaning_tnm, check_rm_tnm = get_tnm_phrase(df=df, col='report_text_anon', 
                                                                                  remove_unusual=True, 
                                                                                  remove_historical=False, 
                                                                                  remove_falsepos=True)

# Add nearby tumour keywords (can help decide which tumour the TNM phrase refers to, if needed)
matches_tnm = add_tumour_tnm(df, matches_tnm, col_report='report_text_anon', targetcol='target_before_clean')

# Get TNM values from phrases
df, check_values_tnm = get_tnm_values(df, matches=matches_tnm, col='report_text_anon', pathology_prefix=False)

# Mark all reports that have T, N or M values
mask = ~(df['T'].isna() & df['N'].isna() & df['M'].isna())
df['has_tnm'] = 0
df.loc[mask, 'has_tnm'] = 1
print('Number of reports with and without T, N or M value according to code:\n\n{}'.format(df.has_tnm.value_counts()))

# Get excluded TNM matches & add indicator
# This helps check reports that had some matches marked as invalid
# Contrary to CRC detection, it is useful to check these, as the final result includes max and min of all matches marked as valid
df['false_tnm'] = 0
df['row'] = np.arange(df.shape[0])
#row_false = np.setdiff1d(check_rm_tnm.row, matches_tnm.row)
row_false = check_rm_tnm.row
df.loc[df.row.isin(row_false), 'false_tnm'] = 1
print(df.groupby(['brc', 'report_type'])['false_tnm'].mean())

# Lil summary
print('--------')
cols = ['T', 'N', 'M']
for c in cols:
    print(c)
    display(df[c].value_counts())
    
n = df.groupby(['brc', 'has_tnm']).size()
ntot = df.groupby('brc').size()
print(n)
print(n/ntot)
print('--------')

# Save to disk for reference
os.chdir(out_path)
#tstamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
dfs   = [df, 
         matches_tnm, check_phrases_tnm, check_cleaning_tnm, check_rm_tnm]
names = ['reports-all_crc-true_tnm-true_recur-false', 
         'tnm-matches', 'tnm-check-phrases', 'tnm-check-cleaning', 'tnm-check-rm']
for n, d in zip(names,dfs):
    fname = n + '.csv'
    print('Saving to file {}...'.format(fname))
    d.to_csv(fname, index=False)

In [ ]:
# Read reports (with CRC status and TNM staging)
read_from_disk=True
if read_from_disk:
    os.chdir(out_path)
    files = os.listdir()
    fname = [f for f in files if f.startswith('reports-all_crc-true_tnm-true_recur-false')][0]
    print(fname)
    df = pd.read_csv(fname)
    display(df.head())

In [ ]:
# Get recurrence and metastasis (about 27 min per 74k reports)
df, matches_rec = get_recurrence(df, 'report_text_anon', verbose=False)

# Mark all reports that have recurrence or metastasis
print(df.recurrence.unique(), df.metastasis.unique())
df['has_recurrence'] = 0
df.loc[~df['recurrence'].isna(), 'has_recurrence'] = 1
df['has_metastasis'] = 0
df.loc[~df['metastasis'].isna(), 'has_metastasis'] = 1
print('Number of reports with recurrence:\n\n{}'.format(df.has_recurrence.value_counts()))
print('Number of reports with metastasis:\n\n{}'.format(df.has_metastasis.value_counts()))

# Add indicator for excluded matches
ex_rec = matches_rec.loc[(matches_rec.exclusion_indicator==1) & (matches_rec.concept=='recurrence')]
print(ex_rec.shape[0])
df['false_recur'] = 0
df['row'] = np.arange(df.shape[0])
df.loc[df.row.isin(ex_rec.row), 'false_recur'] = 1
print(df.groupby(['brc', 'report_type'])['false_recur'].mean())

ex_met = matches_rec.loc[(matches_rec.exclusion_indicator==1) & (matches_rec.concept=='metastasis')]
print(ex_met.shape[0])
df['false_met'] = 0
df['row'] = np.arange(df.shape[0])
df.loc[df.row.isin(ex_met.row), 'false_met'] = 1
print(df.groupby(['brc', 'report_type'])['false_met'].mean())

# Save to disk for reference
os.chdir(out_path)
#tstamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
dfs   = [df, matches_rec]
names = ['reports-all_crc-true_tnm-true_recur-true', 'recur-matches']
for n, d in zip(names,dfs):
    fname = n + '.csv'
    print('Saving to file {}...'.format(fname))
    d.to_csv(fname, index=False)

In [ ]:
# Read reports (with CRC status and TNM staging and recurrence)
read_from_disk=False
if read_from_disk:
    os.chdir(out_path)
    files = os.listdir()
    fname = [f for f in files if f.startswith('reports-all_crc-true_tnm-true_recur-true')][0]
    print(fname)
    df = pd.read_csv(fname)
    display(df.head())

In [ ]:
df.columns

In [ ]:
# Get crm
df, matches_crm, nonmatches_crm = get_crm(df, 'report_text_anon')

In [ ]:
# Get emvi
df, matches_emvi, nonmatches_emvi = get_emvi(df, 'report_text_anon')

In [ ]:
# Double check
print(df.columns)
print(df.shape)

In [ ]:
# Save to disk for reference
os.chdir(out_path)
#tstamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
dfs   = [df, matches_crm, nonmatches_crm, matches_emvi, nonmatches_emvi]
names = ['reports-all_crc-true_tnm-true_recur-true_crmemvi-true', 'crm-matches', 'crm-nonmatches', 'emvi-matches', 'emvi-nonmatches']
for n, d in zip(names,dfs):
    fname = n + '.csv'
    print('Saving to file {}...'.format(fname))
    d.to_csv(fname, index=False)

### 1.3. Double check the results

In [ ]:
files = os.listdir(out_path)
[f for f in files if f.startswith('reports-all_crc-true_tnm-true_recur-true_crmemvi-true')]

In [ ]:
# Read reports
#  NB -- need to have lineterminator='\n' (works with c engine), as otherwise '\r' interpreted as lineterminator too
#usecols = ['brc', 'subject_id', 'row', 'imaging_date', 'report_date', 'imaging_code',
#           'report_text_anon', 'report_type', 'crc_nlp', 'has_tnm', 'has_recurrence', 'has_metastasis',
#           'false_tnm', 'false_recur', 'T', 'T_sub', 'T_min', 'T_sub_min', 'N', 'N_sub']
os.chdir(out_path)
files = os.listdir()
#fname = [f for f in files if f.startswith('reports-all_crc-true_tnm-true_recur-true')][0]
fname = 'reports-all_crc-true_tnm-true_recur-true_crmemvi-true.csv'
print('Reading from file {}'.format(fname))
#df = pd.read_csv(fname, usecols=None, engine='c', sep=',', lineterminator='\n')
df = pd.read_csv(fname)

print('\nColumns: {}'.format(df.columns))
print('Shape: {}'.format(df.shape))
print('Proportion of reports with crc ({:.2f}), recurrence ({:.2f}), tnm ({:.2f}), metastasis ({:.2f})'.format(\
       df.crc_nlp.mean(), df.has_recurrence.mean(), df.has_tnm.mean(), df.has_metastasis.mean()))
print('Unique values for BRC (dummy checking read csv):{}'.format(df.brc.unique()))
#display(df.head())

In [ ]:
# Counts
def count(df, vcol, gcols=['report_type']):
    st = df.groupby(gcols)[vcol].size().rename('n')
    s0 = df.groupby(gcols)[vcol].sum().rename('count')
    s1 = df.groupby(gcols)[vcol].mean().round(3).rename('percent')
    s1 *= 100
    s = pd.concat(objs=[st, s0, s1], axis=1)
    return s



In [ ]:
cols = ['crc_nlp', 'false_crc_nlp']
for vcol in cols:
    print('\n---{}'.format(vcol))
    print(count(df, vcol, ['report_type']))

In [ ]:
cols = ['has_tnm', 'false_tnm']
for vcol in cols:
    print('\n---{}'.format(vcol))
    print(count(df, vcol, ['report_type']))

In [ ]:
cols = ['has_recurrence', 'false_recur']
for vcol in cols:
    print('\n---{}'.format(vcol))
    print(count(df, vcol, ['report_type']))

In [ ]:
cols = ['has_metastasis', 'false_met']
for vcol in cols:
    print('\n---{}'.format(vcol))
    print(count(df, vcol, ['report_type']))

In [ ]:
# Check matches for pathology future - why does it seem to have more false tnm?
# Seems that certain reporting format is used more
fname = 'tnm-check-rm.csv'
print('Reading from file {}'.format(fname))
#df = pd.read_csv(fname, usecols=None, engine='c', sep=',', lineterminator='\n')
matches = pd.read_csv(fname)
print(matches.shape, matches.columns)
print(df.shape[0])

df['row'] = np.arange(df.shape[0])

mask = (df.report_type == 'pathology_future') & (df.false_tnm == 1)
rows = df.loc[mask, 'row']
m = matches.loc[matches.row.isin(rows)]
print(m.shape, m.row.nunique())
m[['left', 'target', 'right', 'exclusion_reason']].drop_duplicates(subset=['target'])

In [ ]:
# Check matches for imaging - why does it seem to have so many false crc?
# Seems that certain reporting format is used more
fname = 'matches-crc.csv'
print('Reading from file {}'.format(fname))
#df = pd.read_csv(fname, usecols=None, engine='c', sep=',', lineterminator='\n')
matches = pd.read_csv(fname)
print(matches.shape, matches.columns)
print(df.shape[0])

mask = (df.report_type == 'imaging') & (df.false_crc_nlp == 1)
rows = df.loc[mask, 'row']
m = matches.loc[matches.row.isin(rows)]
print(m.shape, m.row.nunique())
m[['left', 'target', 'right', 'exclusion_reason']].drop_duplicates(subset=['target'])

In [ ]:
"""
# Mark reports that were previously labelled and used multiple times in code development - excluding these just in case
os.chdir(data_path)
files = os.listdir()
files = [f for f in files if f.startswith('reports-labelled')]
print(files)

ex = pd.DataFrame()
for f in files:
    e = pd.read_csv(f, engine='c', sep=',', lineterminator='\n')
    if 'date' in e.columns:
        e = e.rename(columns={'date':'report_date'})
    e = e.rename(columns={'subject':'subject_id', 'safe_report':'report_text_anon'})
    e = e[['subject_id', 'report_text_anon', 'report_date', 'report_type']]
    print(e.shape, e.columns)
    ex = pd.concat(objs=[ex, e], axis=0)
ex = ex.drop_duplicates(subset=['report_text_anon'])
print(ex.shape)

# For consistency
ex.report_text_anon = ex.report_text_anon.str.replace('\r', '\n\n')
ex.report_text_anon = ex.report_text_anon.str.replace('”', '\n\n')

# Dbl check - unmatched reports mostly endoscopy, though 12 img reports - not sure why
mask = ex.report_text_anon.isin(df.report_text_anon)
test = ex.loc[~mask]
print(test.report_type.value_counts())
test = test.loc[test.report_type=='imaging_relevant']
test = test.merge(df.rename(columns={'report_text_anon': 'r'})[['subject_id', 'report_date', 'r']], how='left')
print(test.shape)

# Remove
print('---')

mask = df.report_text_anon.isin(ex.report_text_anon)
print(mask.sum())
print(df.shape)
df = df.loc[~mask]
print(df.shape)
print(df.crc_nlp.mean(), df.has_recurrence.mean(), df.has_tnm.mean(), df.has_metastasis.mean())
print(df.crc_nlp.sum(), df.has_recurrence.sum(), df.has_tnm.sum(), df.has_metastasis.sum())
print('-----')
"""


In [ ]:
ex = pd.read_csv(os.path.join(out_path, 'recur-matches.csv'))

# Dbl check the large proportion of false recur matches in imaging reports 
# Seems that many are negated, but many also historic
# Currently, matches preceded by 'clinical information' are classified historic
t = ex.loc[ex.row.isin(df.loc[(df.report_type=='imaging')&(df.brc=='RMH')].row)]
print(t.shape)
print(t.exclusion_reason.value_counts())
tsub = t.sample(100, random_state=42)[['row','left', 'target', 'right', 'exclusion_reason']]

pd.set_option('display.max_colwidth', 500, 'display.min_rows', 20, 'display.max_rows', 20)
display(tsub)

display(tsub.loc[tsub.exclusion_reason.fillna('').str.contains('historic')])

In [ ]:
# Check how many reports have both CRC, TNM and recurrence 
#  Doesn't seem there's large overlap + recur needs to be extracted from reports that are not directly CRC reports
df[['crc_nlp', 'has_tnm', 'has_recurrence']].value_counts().reset_index()

In [ ]:
# Check how many reports have CRC and TNM
df[['crc_nlp', 'has_tnm']].value_counts().reset_index()

In [ ]:
# Dbl check report counts 
s = df[['brc', 'report_type', 'crc_nlp', 'has_tnm']].value_counts().rename('n').reset_index()
s = s.sort_values(['brc', 'report_type', 'crc_nlp', 'has_tnm'])

pd.set_option('display.max_colwidth', 500, 'display.min_rows', 50, 'display.max_rows', 50)
s

In [ ]:
# Dbl check report counts 
s = df[['brc', 'report_type', 'crc_nlp', 'has_tnm', 'false_tnm']].value_counts().rename('n').reset_index()
s = s.sort_values(['brc', 'report_type', 'crc_nlp', 'has_tnm', 'false_tnm'])

pd.set_option('display.max_colwidth', 500, 'display.min_rows', 50, 'display.max_rows', 50)
s

In [ ]:
# Dbl check reports where min and max T-stage differ
"""
cols = ['T_pre_indecision', 'T_indecision',
       'T_sub_indecision', 'N_indecision', 'N_sub_indecision', 'M_indecision',
       'M_sub_indecision']
df[cols].mean(axis=0)
"""

In [ ]:
for i in range(10):
    print('\n====')
    row = dfsub.iloc[i]
    print(row['T'], row['T_min'])
    print(row.report_text_anon)

In [ ]:
# Dbl check report counts 
s = df[['brc', 'report_type', 'has_recurrence', 'has_metastasis']].value_counts().rename('n').reset_index()
s = s.sort_values(['brc', 'report_type'])

pd.set_option('display.max_colwidth', 500, 'display.min_rows', 50, 'display.max_rows', 50)
s

## 2. Select reports

### 2.1. Select reports for TNM and CRC (2023-05-14)

---- Newer report selection strategy ----

Set 1
* TNM : [OXFORD] x [img, path] x [has_tnm, ~has_tnm] -> 4 categories -> 400 reports
* CRC : [OXFORD] x [img, path] x [has_crc, ~has_crc] -> 4 categories -> 400 reports

Set 2
* TNM : [OXFORD_FUTURE] x [img, path] x [has_tnm, ~has_tnm] -> 4 categories -> 400 reports
* CRC : [OXFORD_FUTURE] x [img, path] x [has_crc, ~has_crc] -> 4 categories -> 400 reports

In [ ]:
# Read reports
#  NB -- in previous version, needed to have lineterminator='\n' (works with c engine), as otherwise '\r' interpreted as lineterminator too
#usecols = ['brc', 'subject_id', 'row', 'imaging_date', 'report_date', 'imaging_code',
#           'report_text_anon', 'report_type', 'crc_nlp', 'has_tnm', 'has_recurrence', 'has_metastasis',
#           'false_tnm', 'false_recur', 'T', 'T_sub', 'T_min', 'T_sub_min', 'N', 'N_sub']
os.chdir(out_path)
files = os.listdir()
#fname = [f for f in files if f.startswith('reports-all_crc-true_tnm-true_recur-true')][0]
fname = 'reports-all_crc-true_tnm-true_recur-true_crmemvi-true.csv'
print('Reading from file {}'.format(fname))
#df = pd.read_csv(fname, usecols=None, engine='c', sep=',', lineterminator='\n')
df = pd.read_csv(fname)

print('\nColumns: {}'.format(df.columns))
print('Shape: {}'.format(df.shape))
print('Proportion of reports with crc ({:.2f}), recurrence ({:.2f}), tnm ({:.2f}), metastasis ({:.2f})'.format(\
       df.crc_nlp.mean(), df.has_recurrence.mean(), df.has_tnm.mean(), df.has_metastasis.mean()))
print('Unique values for BRC (dummy checking read csv):{}'.format(df.brc.unique()))
#display(df.head())

In [ ]:
print(df.columns)

In [ ]:
# if min-max values are the same, retain max only
cols = ['T_pre', 'T', 'N', 'M', 'V', 'R', 'L', 'Pn', 'SM', 'H', 'G', 'CRM', 'EMVI']
cols_min = [c + '_min' for c in cols]
for c, cmin in zip(cols, cols_min):
    print('--')
    print(c, cmin)

    mask = df[c] == df[cmin]
    print(mask.sum())
    #print(df.loc[mask, [c, cmin]])
    df.loc[mask, cmin] = np.nan
    #print(df.loc[mask, [c, cmin]])


In [ ]:
def select(dfsub, n_select, rng):

    # Randomly sample indices
    n = dfsub.shape[0]
    #n_select = np.floor(p_select*n).astype(int)
    c = n_select/n*100
    if n < n_select:
        i = np.arange(n)
    else:
        i = rng.choice(n, n_select, replace=False)
    
    # Retain sampled indices
    return dfsub.iloc[i,:], c

In [ ]:
print(df[['brc', 'report_type']].drop_duplicates())

In [ ]:
seed = 42
rng = np.random.default_rng(seed=seed)
n_select = 100
#p_select  = 0.05
dfa = pd.DataFrame()
dfb = pd.DataFrame()
#dfc = pd.DataFrame()
suma  = pd.DataFrame()
sumb  = pd.DataFrame()

# ---- REPORTS WITH AND WITHOUT TNM ----

# Oxford reports
for has_tnm in [0, 1]:
    for report_type in ['pathology', 'imaging']:
        for brc in ['OXFORD']:

            # Subset the data
            mask = (df.has_tnm == has_tnm) & (df.report_type == report_type) & (df.brc == brc)
            dfsub = df.loc[mask].copy()
            n = dfsub.shape[0]
            print(n, n_select)
            dfsub, c = select(dfsub, n_select, rng)
            dfa = pd.concat(objs=[dfa, dfsub], axis=0)

            # Summarise
            s = pd.DataFrame([[brc, dfsub.has_tnm.mean(), report_type, dfsub.crc_nlp.mean(), dfsub.false_tnm.mean(), n, n_select, seed, c]])
            s.columns = ['brc', 'has_tnm', 'report_type', 'crc_nlp', 'false_tnm', 'n', 'n_select', 'seed', 'coverage (%)']
            suma = pd.concat(objs=[suma, s], axis=0)


# ---- REPORTS WITH AND WITHOUT CRC ----

# Oxford reports
for crc_nlp in [0, 1]:
    for report_type in ['pathology', 'imaging']:
        for brc in ['OXFORD']:

            # Subset the data
            mask = (df.crc_nlp == crc_nlp) & (df.report_type == report_type) & (df.brc == brc)
            dfsub = df.loc[mask].copy()
            n = dfsub.shape[0]
            print(n, n_select)
            dfsub, c = select(dfsub, n_select, rng)
            dfb = pd.concat(objs=[dfb, dfsub], axis=0)

            # Summarise
            s = pd.DataFrame([[brc, dfsub.has_tnm.mean(), report_type, dfsub.crc_nlp.mean(), dfsub.false_tnm.mean(), n, n_select, seed, c]])
            s.columns = ['brc', 'has_tnm', 'report_type', 'crc_nlp', 'false_tnm', 'n', 'n_select', 'seed', 'coverage (%)']
            sumb = pd.concat(objs=[sumb, s], axis=0)





In [ ]:
suma

In [ ]:
sumb

In [ ]:
df.crc_nlp.mean()

In [ ]:
## Dbl check logic of selecting OUH reports
dfox = df.loc[(df.brc == 'OXFORD') & (df.report_type.isin(['imaging', 'pathology']))]
print(dfox.report_type.unique(), dfox.brc.unique())

# 16% have TNM predicted by alg, makes sense to stratify by TNM
print(dfox.has_tnm.mean())

# 29% have CRC according to algorithm, makes sense to stratify
print(dfox.crc_nlp.mean())

In [ ]:
## Dbl check logic of selecting OUH reports: 16% have TNM predicted by alg, makes sense to stratify by TNM
s = dfox.groupby('has_tnm')['crc_nlp'].value_counts(normalize=True, sort=False)
print(s)

s = dfox.groupby('crc_nlp')['has_tnm'].value_counts(normalize=True, sort=False)
print(s)

s = dfox.groupby('has_tnm')['crc_nlp'].value_counts(normalize=False, sort=False)
print(s)

s = dfox.groupby('crc_nlp')['has_tnm'].value_counts(normalize=False, sort=False)
print(s)

"""Note

When selecting randomly from has_tnm subgroups,
about 17 of 100 reports will have no TNM but CRC

When selecting randomly from crc_nlp subgroups,
about 52 of 100 will have no TNM but CRC

Estimate of PPV skewed? [AT comment 2025-07-21: estimate of PPV could be skewed 
if evaluating the TNM and CRC algorithms on the same set of reports, not separately as done in this notebook.]
"""


In [ ]:
df.report_type

In [ ]:
dfsub = df.loc[(df.crc_nlp == 1) & (df.has_tnm == 0)]

dfsub.report_text_anon.iloc[10]

In [ ]:
# Dbl check
s = dfa.groupby(['brc', 'has_tnm', 'report_type', 'crc_nlp']).size().rename('n_select').reset_index()
s = s.sort_values(['brc', 'crc_nlp', 'has_tnm'])
s

In [ ]:
# Dbl check
s = dfb.groupby(['brc', 'has_tnm', 'report_type', 'crc_nlp']).size().rename('n_select').reset_index()
s = s.sort_values(['brc', 'report_type', 'crc_nlp', 'has_tnm'])
s

In [ ]:
# Dbl check false_tnm proportion
dfa.groupby(['brc', 'report_type', 'crc_nlp', 'has_tnm'])['false_tnm'].value_counts().rename('n').reset_index()

In [ ]:
# Dbl check there are no duplicates
print(dfa.shape)
print(dfa.drop_duplicates(subset=['report_text_anon']).shape)

print(dfb.shape)
print(dfb.drop_duplicates(subset=['report_text_anon']).shape)


In [ ]:
# Sort reports randomly
dfa_sort = dfa.sample(n=dfa.shape[0], random_state=42, replace=False)
print(dfa_sort.drop_duplicates().shape[0] == dfa_sort.shape[0])

dfb_sort = dfb.sample(n=dfb.shape[0], random_state=42, replace=False)
print(dfb_sort.drop_duplicates().shape[0] == dfb_sort.shape[0])



In [ ]:
# Save (duplicates - orig file, and file to be modified by checking labels)
#tstamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
fnames = ['set1_tnm.csv', 'set1_tnm_labelled.csv']
for fname in fnames:
    print('Saving to {}'.format(fname))
    dfa_sort.to_csv(out_path / fname, index=False)

    # Dbl check that file can be read 
    test = pd.read_csv(out_path / fname) #, engine='c', lineterminator='\n')
    print(test.brc.unique())


# Save
#tstamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
fnames = ['set1_crc.csv', 'set1_crc_labelled.csv']
for fname in fnames:
    print('Saving to {}'.format(fname))
    dfb_sort.to_csv(out_path / fname, index=False)

    # Dbl check that file can be read 
    test = pd.read_csv(out_path / fname) #, engine='c', lineterminator='\n')
    print(test.brc.unique())


In [ ]:
## Dbl check T, N, M proportion separately in set1
df = pd.read_csv(out_path / 'set1_tnm.csv')
print(df.shape)

cols = ['T', 'N', 'M']
for c in cols:
    test = (~df[c].isna()).sum()
    print('{}: {}, unique: {}'.format(c, test, df[c].unique().tolist()))


In [ ]:
## Dbl check T, N, M proportion separately in set2
df = pd.read_csv(out_path / 'set2_tnm.csv')
print(df.shape)

cols = ['T', 'N', 'M']
for c in cols:
    test = (~df[c].isna()).sum()
    print('{}: {}, unique: {}'.format(c, test, df[c].unique().tolist()))


## Additional. How many reports to select?

In [ ]:
# What is the probability that p_hat differs from p by more than e, given sample size n?
n = 100
e = 0.05

# Assume that asymptotic distribution of p_hat-p=delta ~ N(0, p(1-p)/n)

#----
# Method 1 
#----

## Assume fixed population proportion p
## 1-cdf - probability that delta greater than e
## *2, as can also be smaller than e
p    = 0.9
se   = np.sqrt(p*(1-p)/n)
prob = (1 - norm.cdf(x=e, loc=0, scale=se))*2
print('\nFor n={}, probability that p_hat differs from p by more than {}: {}'.format(n, e, prob))

## ppf - inverse of cdf
alpha=0.05
q = norm.ppf((1-alpha/2), loc=0, scale=1)
print(q)
ci = [p-q*se, p+q*se]
print('{}% ci: {}'.format((1-alpha)*100, ci))
#print('\nn={} helps ensure that probability that p_hat differs from p by more than {} is {}'.format(n, e, prob_desired))

#----
# Method 2 
#----

## Assume population proportion p is drawn from truncated normal 
## with mu=p, std=0.05, and bounded in [0,1]
myclip_a = 0
myclip_b = 1
my_mean  = p
my_std   = 0.05
a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
x = np.linspace(0, 1, 1000)
y = truncnorm.pdf(x, a=a, b=b, loc=my_mean, scale=my_std)
plt.plot(x, y)
print('\nPrior')
plt.show()

## Simulate p from truncated normal
## For each value of p, simulate delta from N(0, se_hat)
nsim  = 100000  # Number of simulations
psim  = truncnorm.rvs(a=a, b=b, loc=my_mean, scale=my_std, size=nsim) # Simulate p from prior
se    = np.sqrt(psim*(1-psim)/n) # Derive se
delta = norm.rvs(loc=0, scale=se, size=nsim) # Simulate delta

mask = np.abs(delta) > e
prob = mask.sum()/len(mask)  # Estimate probability
print('\nFor n={}, Bayesian probability that p_hat differs from p by more than {}: {}'.format(n, e, prob))

In [ ]:
## CIs for population proportion
from statsmodels.stats.proportion import proportion_confint

n = 100
successes = int(0.9*n)
ci = proportion_confint(count=successes, nobs=n, alpha=0.05, method='normal')
print(ci)
ci = proportion_confint(count=successes, nobs=n, alpha=0.05, method='wilson')
print(ci)

In [ ]:
# Double check that entered data corresponds to labeller.R display
"""
os.chdir(out_path)
f = 'reports-labelled-tnm_20210803_1710.csv'
df = pd.read_csv(f)

for i in range(30):
    print('---{}---'.format(i+1))
    print(df.iloc[i])
"""